# Pickhardt Payments Example

Example code demonstrating how to use the `pickhardtpayments` package in python. You need to install the library via `pip install pickhardtpayments` or you can download the full source code at https://ln.rene-pickhardt.de or a copy from github at: https://www.github.com/renepickhardt/pickhardtpayments

Of course you can use the classes in the library to create your own async payment loop or you could exchange the Oracle to talk to the actual Lightning network by wrapping against your favourite node implementation and exposing the the `send_onion` call. 

This example assumes a randomly generated Oracle to conduct payments in a simulated way. For this you will need an actual channelgraph which you can get for example with `lightning-cli listchannels > listchannels20220412.json`

## Acknowledgements & Funding
This work is funded via various sources including NTNU & BitMEX as well as many generous donors via https://donate.ln.rene-pickhardt.de or https://www.patreon.com/renepickhardt Feel free to go to my website at https://ln.rene-pickhardt.de to learn how I have been contributing to the open source community and why it is important to have independent open source contributors. In case you also wish to support me I will be very grateful

In [1]:
from pickhardtpayments.ChannelGraph import ChannelGraph
from pickhardtpayments.UncertaintyNetwork import UncertaintyNetwork
from pickhardtpayments.OracleLightningNetwork import OracleLightningNetwork
from pickhardtpayments.SyncSimulatedPaymentSession import SyncSimulatedPaymentSession

In [2]:
#Rene Pickhardt's public node key
RENE = "03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df"
#Carsten Otto's public node key
C_OTTO = "027ce055380348d7812d2ae7745701c9f93e70c1adeb2657f053f91df4f2843c71"

#we first need to import the chanenl graph from core lightning jsondump
#you can get your own data set via:
# $: lightning-cli listchannels > listchannels20220412.json
# alternatively you can go to https://ln.rene-pickhardt.de to find a data dump
channel_graph = ChannelGraph("listchannels20220412.json")

#we now create an Oracle. This is a Simulated Network that assumes uniform distribution 
#of the liquidity for the channels on the `channel_graph`. Of course one could create one's own 
#oracle (for example a wrapper to an existing lightning network node / implementation)
oracle_lightning_network = OracleLightningNetwork(channel_graph)

In [3]:
# Since we randomly generated our Oracle but also since we control it we can compute
# The maximum possible amount that can be payed between two nodes
maximum_payable_amount =oracle_lightning_network.theoretical_maximum_payable_amount(RENE,C_OTTO,1000)
print(maximum_payable_amount, "sats would be possible on this oracle to deliver if including 1 sat basefee channels")


107179625 sats would be possible on this oracle to deliver if including 1 sat basefee channels


In [4]:
#Of course we want to restrict ourselves to the zeroBaseFee part of the network.
#Therefore we compute the theoretical maximum payable amount for that subgraph
maximum_payable_amount =oracle_lightning_network.theoretical_maximum_payable_amount(RENE,C_OTTO,0)
print(maximum_payable_amount, "sats possible on this oracle on the zeroBaseFeeGraph")

55795527 sats possible on this oracle on the zeroBaseFeeGraph


In [5]:
# We choose an amount that is half the theoretical maximum to demonstrate the
# minimum cost flow solver with Bayesian updates on the Uncertainty Network finds the
# liquidity rather quickly
tested_amount = int(maximum_payable_amount/2)

# From the channel graph we can derive our initial Uncertainty Network which is the main data structure
# that we maintain in order to deliver sats from one node to another
uncertainty_network = UncertaintyNetwork(channel_graph)

#we create a payment session which in this case operates by sending out the onions
#sequentially 
payment_session = SyncSimulatedPaymentSession(oracle_lightning_network, 
                                 uncertainty_network,
                                 prune_network=False)

#we need to make sure we forget all learnt information on the Uncertainty Nework
payment_session.forget_information()

#we run the simulation of pickhardt payments and track all the results
payment_session.pickhardt_pay(RENE,C_OTTO, tested_amount,mu=0,base=0)

Round number:  1
Try to deliver 27897763 satoshi:

Statistics about 10 candidate onions:

successful attempts:
--------------------
 p =  80.89% amt:   1000000 sats  hops: 3 ppm:  1897
 p =  93.93% amt:    356886 sats  hops: 3 ppm:  1973
 p =  54.14% amt:   3000000 sats  hops: 5 ppm:  2580
 p =  46.45% amt:    969011 sats  hops: 5 ppm:  2057

failed attempts:
----------------
 p =  25.52% amt:  10066329 sats  hops: 3 ppm:  2036 
 p =  77.91% amt:    796094 sats  hops: 4 ppm:  2059 
 p =  50.98% amt:   3355443 sats  hops: 4 ppm:  2496 
 p =  42.69% amt:   3051434 sats  hops: 5 ppm:  2551 
 p =  29.38% amt:   5000000 sats  hops: 6 ppm:  5318 
 p =  35.76% amt:    302566 sats  hops: 7 ppm:  3251 

Attempt Summary:

Tried to deliver   27897763 sats
expected to deliver   10997273 sats 	(39.42%)
actually deliverd    5325897 sats 	(19.09%)
deviation: 0.48
planned_fee: 78220.517 sat
paid fees: 12335.715 sat
Runtime of flow computation: 0.49 sec 


Round number:  2
Try to deliver 22571866 satos

## Optimizing for Fees

controlling mu we can decide how much we wish to focus on lower fees. However ,we will see that it will be much harder to deliver the same amount in the sense that we need to send out more onions and also have more failed attampts. Consequently we expect to need more time.

In [6]:
#we need to make sure we forget all learnt information on the Uncertainty Nework
payment_session.forget_information()

#we run the simulation of pickhardt payments and track all the results
payment_session.pickhardt_pay(RENE,C_OTTO, tested_amount,mu=100,base=0)

Round number:  1
Try to deliver 27897763 satoshi:

Statistics about 14 candidate onions:

successful attempts:
--------------------
 p =  36.00% amt:   2000000 sats  hops: 2 ppm:  1360
 p =  22.22% amt:    998574 sats  hops: 4 ppm:  1367
 p =  61.34% amt:    600000 sats  hops: 5 ppm:  1375

failed attempts:
----------------
 p =  70.75% amt:   1000000 sats  hops: 3 ppm:  1352 
 p =  19.37% amt:   4906248 sats  hops: 3 ppm:  1365 
 p =  35.39% amt:    740000 sats  hops: 3 ppm:  1381 
 p =  43.15% amt:   2000000 sats  hops: 4 ppm:  1376 
 p =  20.72% amt:   1000000 sats  hops: 4 ppm:  1379 
 p =  14.17% amt:   4200000 sats  hops: 5 ppm:  1362 
 p =   3.63% amt:   3452941 sats  hops: 5 ppm:  1352 
 p =   5.84% amt:   1000000 sats  hops: 6 ppm:  1354 
 p =   0.19% amt:   4000000 sats  hops: 6 ppm:  1362 
 p =   0.00% amt:   1117059 sats  hops: 6 ppm:  1361 
 p =  40.77% amt:    882941 sats  hops: 6 ppm:  1373 

Attempt Summary:

Tried to deliver   27897763 sats
expected to deliver    54466


Statistics about 9 candidate onions:

successful attempts:
--------------------
 p =  76.33% amt:   1000000 sats  hops: 3 ppm:  1366
 p =  58.03% amt:    178216 sats  hops: 3 ppm:  1387
 p =  55.32% amt:    300000 sats  hops: 5 ppm:  1523
 p =  71.23% amt:     76278 sats  hops: 5 ppm:  1558
 p =  75.21% amt:    122880 sats  hops: 5 ppm:  1374

failed attempts:
----------------
 p =  64.18% amt:    324000 sats  hops: 3 ppm:  1506 
 p =  78.50% amt:    100902 sats  hops: 3 ppm:  1378 
 p =  61.99% amt:    113806 sats  hops: 4 ppm:  1360 
 p =  44.63% amt:    698778 sats  hops: 5 ppm:  1594 

Attempt Summary:

Tried to deliver    2914860 sats
expected to deliver    1848980 sats 	(63.43%)
actually deliverd    1677374 sats 	(57.55%)
deviation: 0.91
planned_fee: 4254.657 sat
paid fees: 2358.134 sat
Runtime of flow computation: 0.48 sec 


Round number:  7
Try to deliver 1237486 satoshi:

Statistics about 4 candidate onions:

successful attempts:
--------------------
 p =  79.12% amt:    324